In [ ]:
from vivarium import Artifact, InteractiveContext
import pandas as pd, numpy as np, os

In [ ]:
! pip list | grep vivarium

# [software verion + hash . date]

In [ ]:
! pip freeze | grep vivarium

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from vivarium import InteractiveContext, Artifact

In [ ]:
import vivarium_gates_mncnh
from vivarium.framework.configuration import build_model_specification
from pathlib import Path

path = Path(vivarium_gates_mncnh.__file__).parent / 'model_specifications/model_spec.yaml'
custom_model_specification = build_model_specification(path)
del custom_model_specification.configuration.observers
# custom_model_specification.configuration.input_data.input_draw_number = 60
custom_model_specification.configuration.population.population_size = 20_000 * 10

artifact_path = custom_model_specification.configuration.input_data.artifact_path
art = Artifact(artifact_path)

artifact_path

In [ ]:
draw_num = custom_model_specification.configuration.input_data.input_draw_number
draw = 'draw_' + str(draw_num)
draw

In [ ]:
sim = InteractiveContext(custom_model_specification)

In [ ]:
sim_step_name = sim.list_components()['risk_factor.hemoglobin']._sim_step_name
sim_step_name()

In [ ]:
def get_pop_with_pregnancy_duration(sim):
    pop = sim.get_population()
    return pd.concat([
        pop,
        sim.get_value('pregnancy_duration')(pop.index)
    ], axis=1)[['pregnancy_outcome', 'gestational_age_exposure', 'pregnancy_duration']]

In [ ]:
pop = get_pop_with_pregnancy_duration(sim)
pregnancy_duration = pop.pregnancy_duration
assert ((pop.pregnancy_outcome == 'partial_term') == pregnancy_duration.notnull()).all(), "pregnancy duration not defined in exactly abortion/miscarriage/ectopic pregnancies pre-ultrasound"
# https://vivarium-research.readthedocs.io/en/latest/models/other_models/pregnancy/gbd_2021_mncnh/index.html#assign-gestational-age-at-end-of-pregnancy-abortion-miscarriage-ectopic-pregnancies
assert (pop[pop.pregnancy_outcome == 'partial_term'].pregnancy_duration >= pd.Timedelta(days=(6 * 7))).all(), "abortion/miscarriage/ectopic pregnancies with duration less than 6 weeks"
assert (pop[pop.pregnancy_outcome == 'partial_term'].pregnancy_duration < pd.Timedelta(days=(24 * 7))).all(), "abortion/miscarriage/ectopic pregnancies with duration greater than 24 weeks"

In [ ]:
while sim_step_name() != "ultrasound":
    pop = get_pop_with_pregnancy_duration(sim)
    assert pop.gestational_age_exposure.isnull().all(), "gestational age assigned pre-ultrasound"
    assert pop.pregnancy_duration.equals(pregnancy_duration), "pregnancy duration changed pre-ultrasound"
    sim.step()

# Go past ultrasound step
sim.step()

In [ ]:
pop = get_pop_with_pregnancy_duration(sim)
pop

In [ ]:
assert pop.pregnancy_duration[pop.pregnancy_outcome == 'partial_term'].equals(pregnancy_duration[pop.pregnancy_outcome == 'partial_term']), "pregnancy duration changed for abortion/miscarriage/ectopic pregnancies"

In [ ]:
assert np.allclose(
    pop[pop.pregnancy_outcome != 'partial_term'].pregnancy_duration / pd.Timedelta(days=7),
    pop[pop.pregnancy_outcome != 'partial_term'].gestational_age_exposure
), "pregnancy duration and gestational age disagree!"

In [ ]:
assert (
    sim.get_value('gestational_age.birth_exposure')(pop.index)
    ==
    pop.gestational_age_exposure
).all(), "gestational age disagrees between value pipeline and state table!"

In [ ]:
(pop[pop.pregnancy_outcome == 'live_birth'].pregnancy_duration / pd.Timedelta(days=7)).describe()

In [ ]:
(pop[pop.pregnancy_outcome == 'stillbirth'].pregnancy_duration / pd.Timedelta(days=7)).describe()

In [ ]:
# The 'risk specific shift' is the amount to delete
# https://github.com/ihmeuw/vivarium_gates_mncnh/blob/bdffaa88cb94eaa53a97df75be2e307dfd52c25e/src/vivarium_gates_mncnh/data/loader.py#L1326-L1335
ifa_ga_deletion = art.load('risk_factor.iron_folic_acid_supplementation.risk_specific_shift')
assert (ifa_ga_deletion.groupby('affected_entity').nunique() == 1).all().all()
ifa_ga_deletion = ifa_ga_deletion.groupby('affected_entity').first().loc['gestational_age'][draw]
ifa_ga_deletion

In [ ]:
# https://vivarium-research.readthedocs.io/en/latest/models/risk_exposures/low_birthweight_short_gestation/gbd_2021/index.html#converting-gbd-s-categorical-exposure-distribution-to-a-continuous-exposure-distribution
# NOTE: This minimum is on the LBWSG category sampled, but baseline IFA deletion decreases GA
assert (pop[pop.pregnancy_outcome == 'live_birth'].pregnancy_duration >= pd.Timedelta(days=((20 - ifa_ga_deletion) * 7))).all(), "live birth pregnancies with duration less than 20 weeks minus baseline IFA deletion"


In [ ]:
# https://vivarium-research.readthedocs.io/en/latest/models/other_models/pregnancy/gbd_2021_mncnh/index.html#assign-birthweight-and-gestational-age-at-end-of-pregnancy-live-births-and-stillbirths
assert (pop[pop.pregnancy_outcome == 'stillbirth'].pregnancy_duration >= pd.Timedelta(days=((24 - ifa_ga_deletion) * 7))).all(), "stillbirth pregnancies with duration less than 24 weeks minus baseline IFA deletion"